# Demos: Lecture 20

In [ ]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import expm

from lecture20_helpers import *

## Demo 1: commutation chaos

Time evolution under the Hamiltonian

$$
\hat{H} = - Z - X
$$

In [ ]:
H = qml.Hamiltonian([1, 1], [qml.PauliZ(0), qml.PauliX(0)])

def construct_unitary(t):
    return expm(-1j * t * qml.matrix(H))

In [ ]:
dev = qml.device('default.qubit', wires=1)

@qml.qnode(dev)
def change_pauli_order(t, exact=False, reverse=False):
    qml.Hadamard(wires=0)

    if exact:
        qml.QubitUnitary(construct_unitary(t), wires=0)
    else:
        if reverse:
            qml.evolve(qml.PauliX(0), t)
            qml.evolve(qml.PauliZ(0), t)
        else:
            qml.evolve(qml.PauliZ(0), t)
            qml.evolve(qml.PauliX(0), t)
    
    return qml.expval(qml.PauliY(wires=0))

In [ ]:
times = np.linspace(0, 10, 100)

results_exact = [change_pauli_order(t, exact=True) for t in times]
results = [change_pauli_order(t) for t in times]
results_reversed = [change_pauli_order(t, reverse=True) for t in times]

plt.figure(figsize=(10, 6))
plt.plot(times, results_exact, label="True")
plt.plot(times, results, label="Original")
plt.plot(times, results_reversed, label="Reversed")
plt.legend(fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Evolution time", fontsize=14)
plt.ylabel("<Y>", fontsize=14)
plt.show()

## Demo 2: Trotterization

In [ ]:
dev = qml.device('default.qubit', wires=1)

@qml.compile(pipeline=[], basis_set=["RX", "RY", "RZ", "Hadamard", "CNOT"])
@qml.qnode(dev)
def trotterize(t, exact=False, trotter_steps=1):
    qml.Hadamard(wires=0)

    if exact:
        qml.QubitUnitary(construct_unitary(t), wires=0)
    else:
        qml.exp(H, -1j * t, num_steps=trotter_steps)
    
    return qml.expval(qml.PauliY(wires=0))

In [ ]:
print(qml.draw(trotterize)(0.5, exact=False, trotter_steps=3))

In [ ]:
trotter_steps = [1, 2, 5, 10, 20, 50]

results_exact = [trotterize(t, exact=True) for t in times]
results_trotter = [np.array([trotterize(t, trotter_steps=N_T) for t in times]) for N_T in trotter_steps]

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(times, results_exact, label="True", linestyle="--")
for N_T, res in zip(trotter_steps, results_trotter):
    plt.plot(times, res, label=f"{N_T} steps")
plt.legend(fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Evolution time", fontsize=14)
plt.ylabel("<Y>", fontsize=14)
plt.show()

Let's plot the operator error for varying number of Trotter steps.

In [ ]:
t = 1
exact_matrix = trotterize(t, exact=True)

# Take the norm of the difference between the exact/approximate unitary matrices
error = [np.linalg.norm(exact_matrix - trotterize(t, trotter_steps=N_T)) for N_T in trotter_steps]

In [ ]:
plt.plot(trotter_steps, error)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Number of Trotter steps", fontsize=14)
plt.ylabel("Approximation error", fontsize=14)
plt.show()

## Demo 3: time evolution of the deuteron

In [ ]:
def construct_hamiltonian(wires):
    coeffs = [15.532, 0.218, -6.125, -9.625, -2.143, -3.913, -2.143, -3.913]
    
    ops = [
        qml.Identity(wires[0]), 
        qml.PauliZ(wires[0]),
        qml.PauliZ(wires[1]),
        qml.PauliZ(wires[2]),
        qml.PauliX(wires[0]) @ qml.PauliX(wires[1]),
        qml.PauliX(wires[1]) @ qml.PauliX(wires[2]),
        qml.PauliY(wires[0]) @ qml.PauliY(wires[1]),
        qml.PauliY(wires[1]) @ qml.PauliY(wires[2]),
    ]
    
    return qml.Hamiltonian(coeffs, ops)

In [ ]:
H = construct_hamiltonian([0, 1, 2])
print(H)

In [ ]:
eigvals, eigvecs = np.linalg.eigh(qml.matrix(H))

target_eigval = eigvals[0]
target_eigvec = eigvecs[:, 0]

In [ ]:
target_eigval

In [ ]:
target_eigvec

In [ ]:
dev = qml.device('default.qubit', wires=3)

def evolve_deuteron(H, t, num_steps=1):
    qml.exp(H, -1j * t, num_steps=num_steps)

@qml.compile(pipeline=[], basis_set=["RX", "RY", "RZ", "Hadamard", "CNOT"])
@qml.qnode(dev)
def evolve_and_measure(H, t, num_steps=1):
    qml.MottonenStatePreparation(target_eigvec, wires=dev.wires)
    evolve_deuteron(H, t, num_steps)
    return qml.expval(H)

In [ ]:
times = np.linspace(0, 1, 10)

plt.figure(figsize=(10, 6))
plt.axhline(y=target_eigval, color='black', linestyle='--', label="Exact")

for N_T in trotter_steps:
    results = [evolve_and_measure(H, t, num_steps=N_T) for t in times]
    plt.plot(times, results, label=f"{N_T} steps")
    
plt.legend(fontsize=14)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.xlabel("Evolution time", fontsize=14)
plt.ylabel("<H>", fontsize=14)
plt.show()

## Demo 4: run quantum phase estimation

In [ ]:
target_eigval

In [ ]:
target_eigvec

$e^{-iEt} = e^{2\pi i \theta}$

In [ ]:
t = 1
u_eigval = t * target_eigval / (-2 * np.pi)

In [ ]:
u_eigval

In [ ]:
n_estimation_wires = 5
n_target_wires = 3

estimation_wires = list(range(n_estimation_wires))
target_wires = list(range(n_estimation_wires, n_estimation_wires+n_target_wires))

H_QPE = construct_hamiltonian(target_wires)

dev = qml.device("default.qubit", wires=n_estimation_wires+n_target_wires)

In [ ]:
@qml.qnode(dev)
@qml.compile(pipeline=[], basis_set=["RX", "RY", "RZ", "Hadamard", "PhaseShift", "CNOT"], expand_depth=10)
def quantum_phase_estimation(H_QPE, t, num_steps=1):
    # Prepare uniform sup on estimation wires
    for wire in estimation_wires:
        qml.Hadamard(wires=wire)
    
    # Prepare eigenstate on target wires
    qml.MottonenStatePreparation(target_eigvec, wires=target_wires)
    
    # Apply controlled rotations
    for control_wire in estimation_wires:
        evolution_time = t * 2 ** (len(estimation_wires) - control_wire - 1) 
        qml.ctrl(evolve_deuteron, control=control_wire)(H_QPE, evolution_time, num_steps=num_steps)
    
    # Apply inverse QFT to estimation wires
    qml.adjoint(qml.QFT)(wires=estimation_wires)
    
    return qml.probs(wires=estimation_wires)

In [ ]:
print(qml.draw(quantum_phase_estimation)(H_QPE, t))

In [ ]:
num_steps = 1
res = quantum_phase_estimation(H_QPE, t, num_steps=num_steps)

In [ ]:
get_phase_window(res)

In [ ]:
qml.specs(quantum_phase_estimation)(H_QPE, t, num_steps=num_steps)['resources']